# STAT3612 Statistical Machine Learning (2019-20 Semester 1) <a class="tocSkip">

## Assignment 2  <a class="tocSkip">

*ZHANG XINYI UID:3035234571*

## Preparation <a class="tocSkip">

Import necessary packages

In [60]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import scorecardpy as sc
import matplotlib.pyplot as plt
# show plots automatically
%matplotlib inline

from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer,IterativeImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from pygam import LogisticGAM,f,s
from sklearn.linear_model import LinearRegression
from patsy import dmatrix


Load the dataset and preview

In [61]:
df = pd.read_csv('./HelocData.csv', na_values = [-7, -8, -9])
df.head()

RiskFlag    x1     x2   x3     x4    x5   x6   x7     x8    x9  ...   x14  \
0      Bad  75.0  169.0  2.0   59.0  21.0  0.0  0.0  100.0   NaN  ...  36.0   
1      Bad  66.0  502.0  4.0  145.0  34.0  0.0  0.0   97.0  36.0  ...  27.0   
2     Good  69.0  338.0  2.0   62.0  22.0  0.0  0.0   96.0  12.0  ...  35.0   
3     Good  75.0  422.0  1.0   91.0  55.0  0.0  0.0  100.0   NaN  ...  33.0   
4      Bad  63.0  242.0  2.0   68.0  25.0  0.0  0.0  100.0   NaN  ...  19.0   

   x15  x16  x17   x18    x19   x20  x21   x22   x23  
0  NaN  4.0  4.0  43.0  112.0   4.0  6.0   0.0  83.0  
1  4.0  3.0  3.0  80.0   53.0  17.0  3.0  12.0  83.0  
2  0.0  4.0  4.0  25.0  100.0   3.0  2.0   1.0  45.0  
3  0.0  4.0  4.0   2.0   11.0  12.0  2.0   1.0  57.0  
4  NaN  3.0  3.0  73.0    NaN  12.0  1.0   5.0  87.0  

[5 rows x 24 columns]

Load the variable names and preview

In [62]:
data_dict = pd.read_excel('./HelocDataDict.xlsx', index_col=0, squeeze=True)
var_names = data_dict.str.split('(\.| \()', expand=True).iloc[1:,0] # only select the first part
var_names.name = 'Description'

var_names.to_frame()

Description
Variable Names                                                  
x1                          Consolidated version of risk markers
x2                                Months Since Oldest Trade Open
x3                           Months Since Most Recent Trade Open
x4                                        Average Months in File
x5                                    Number Satisfactory Trades
x6                                        Number Trades 60+ Ever
x7                                        Number Trades 90+ Ever
x8                               Percent Trades Never Delinquent
x9                          Months Since Most Recent Delinquency
x10                       Max Delq/Public Records Last 12 Months
x11                                         Max Delinquency Ever
x12                                       Number of Total Trades
x13                      Number of Trades Open in Last 12 Months
x14                                   Percent Installment Trades
x15                      Months Since Most Recent Inq excl 7days
x16                                  Number of Inq Last 6 Months
x17                       Number of Inq Last 6 Months excl 7days
x18                                Net Fraction Revolving Burden
x19                              Net Fraction Installment Burden
x20                         Number Revolving Trades with Balance
x21                       Number Installment Trades with Balance
x22             Number Bank/Natl Trades w high utilization ratio
x23                                  Percent Trades with Balance

### Data set preparation

#### 1) Split the data into training and testing sets with UID as the random seed
#### 2) Convert Risk Flag column to "0" and "1"
#### 3) Use Iterative Imputer to handle missing value
#### 4) Drop variables not interested

In [63]:
# Split the data into training and testing sets with UID as the random seed
np.random.seed(3035234571) 
df_train, df_test = train_test_split(df, test_size=0.2)

train_size = df_train.shape[0]/df.shape[0]
test_size = df_test.shape[0]/df.shape[0]
print('Proportion of training set: {:.2f}%'.format(train_size))
print('Proportion of training set: {:.2f}%'.format(test_size))

Proportion of training set: 0.80%
Proportion of training set: 0.20%


In [64]:
# handling missing value and drop unrelated columns
def df_preprocesser(df, II=None,set='train'):
    df_tmp = df.copy()
    # use Iterative Imputer for imputation
    label_encoder = LabelEncoder()
    df_tmp['RiskFlag'] = label_encoder.fit_transform(df_tmp['RiskFlag'])

    if set=='train':
        II = IterativeImputer().fit(df_tmp)
    df_tmp[:] = II.transform(df_tmp)
    
    #drop unrelated columns
    names = ['RiskFlag','x1', 'x5', 'x16', 'x17', 'x20']
    df_tmp = df_tmp[names]

    return df_tmp, II

In [65]:
#3) Use training data set to fit Iterative Imputer and pass the fitted imputer to testing data.
df_train_clean, II= df_preprocesser(df_train, set='train')
df_test_clean= df_preprocesser(df_test,II, set='test')[0]

/Users/zhangxinyi/anaconda3/lib/python3.7/site-packages/sklearn/impute/_iterative.py:599: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)


In [66]:
#After handling missing value and drop unrelated variables, the data frame looks like:
df_train_clean.head()

RiskFlag    x1    x5  x16  x17   x20
6288        0.0  61.0  26.0  0.0  0.0   7.0
1771        1.0  66.0  33.0  4.0  4.0   2.0
2706        0.0  72.0  56.0  1.0  1.0  12.0
10108       1.0  75.0   7.0  0.0  0.0   2.0
5134        0.0  67.0  31.0  3.0  3.0   6.0

### Question 1

#### 1)Transform data set with the IV binning technique for each selected feature. 
 Use training data to fit the woebin and apply the fitted break knots on testing data


In [67]:
def df_IVBinning(df, x1_breaks=None, x5_breaks=None,x16_breaks=None,
                     x17_breaks=None,x20_breaks=None,set='train'):
    df_tmp = df.copy()
    # use IV binning
    if set=='train':
        x_bins = sc.woebin(df_tmp, y='RiskFlag', method='tree')
        x1_breaks = np.insert(x_bins['x1']['breaks'].values.astype(np.float), 0, -np.inf)
        x5_breaks = np.insert(x_bins['x5']['breaks'].values.astype(np.float), 0, -np.inf)
        x16_breaks = np.insert(x_bins['x16']['breaks'].values.astype(np.float), 0, -np.inf)
        x17_breaks = np.insert(x_bins['x17']['breaks'].values.astype(np.float), 0, -np.inf)
        x20_breaks = np.insert(x_bins['x20']['breaks'].values.astype(np.float), 0, -np.inf)

    df_tmp['x1'] = pd.cut(df_tmp['x1'], bins=x1_breaks, right=True)
    df_tmp['x5'] = pd.cut(df_tmp['x5'], bins=x5_breaks, right=True)
    df_tmp['x16'] = pd.cut(df_tmp['x16'], bins=x16_breaks, right=True)
    df_tmp['x17'] = pd.cut(df_tmp['x17'], bins=x17_breaks, right=True)
    df_tmp['x20'] = pd.cut(df_tmp['x20'], bins=x1_breaks, right=True)
    df_tmp
    
    # one-hot encoding
    df_tmp = pd.get_dummies(df_tmp, columns=['x1','x5','x16','x17','x20'], drop_first=True)

    return df_tmp, x1_breaks, x5_breaks, x16_breaks, x17_breaks, x20_breaks

In [68]:
#Use training data to fit the woebin and apply the fitted break knots on testing data
df_train_IVBinning, x1_breaks, x5_breaks, x16_breaks, x17_breaks, x20_breaks= df_IVBinning(df_train_clean, set='train')
df_test_IVBinning= df_IVBinning(df_test_clean,x1_breaks, x5_breaks, x16_breaks, x17_breaks, x20_breaks, set='test')[0]

[INFO] creating woe binning ...


In [69]:
#After transforming for IV binning method, the data frame looks like:
df_train_IVBinning.head()

RiskFlag  x1_(69.0, 75.0]  x1_(75.0, 84.0]  x1_(84.0, inf]  \
6288        0.0                0                0               0   
1771        1.0                0                0               0   
2706        0.0                1                0               0   
10108       1.0                1                0               0   
5134        0.0                0                0               0   

       x5_(6.0, 12.0]  x5_(12.0, 19.0]  x5_(19.0, 20.0]  x5_(20.0, inf]  \
6288                0                0                0               1   
1771                0                0                0               1   
2706                0                0                0               1   
10108               1                0                0               0   
5134                0                0                0               1   

       x16_(1.0, 2.0]  x16_(2.0, 4.0]  x16_(4.0, inf]  x17_(1.0, 2.0]  \
6288                0               0               0               0   
1771                0               1               0               0   
2706                0               0               0               0   
10108               0               0               0               0   
5134                0               1               0               0   

       x17_(2.0, 4.0]  x17_(4.0, inf]  x20_(69.0, 75.0]  x20_(75.0, 84.0]  \
6288                0               0                 0                 0   
1771                1               0                 0                 0   
2706                0               0                 0                 0   
10108               0               0                 0                 0   
5134                1               0                 0                 0   

       x20_(84.0, inf]  
6288                 0  
1771                 0  
2706                 0  
10108                0  
5134                 0

### Question 1
#### 2)Train the generalized additive model with IV binning technique 
 The variables are all categorical after IV binning transformation

In [70]:
x_train_IV = df_train_IVBinning.iloc[:,1:].values
y_train_IV = df_train_IVBinning.iloc[:,0].values

x_test_IV = df_test_IVBinning.iloc[:,1:].values
y_test_IV = df_test_IVBinning.iloc[:,0].values

from pygam import LogisticGAM,f,s

gam = LogisticGAM(f(0)+f(1)+f(2)+f(3)+f(4)+f(5)+f(6)+f(7)+f(8)+f(9)+f(10)+f(11)+f(12)+f(13)+f(14)+f(15))
# f: factor term
# some parameters combinations in grid search meet the error exception.
gam.gridsearch(x_train_IV,y_train_IV)

                  
y_pred_train_IV = gam.predict(x_train_IV)
y_pred_test_IV = gam.predict(x_test_IV)

# accuracy score
accuracy_train_IV = accuracy_score(y_train_IV, y_pred_train_IV)
accuracy_test_IV = accuracy_score(y_test_IV, y_pred_test_IV)
print('Accuracy on the training set for GAM with IV Binning =', np.round(accuracy_train_IV,4))
print('Accuracy on the test set for GAM with IV Binning =', np.round(accuracy_test_IV,4))

100% (11 of 11) |########################| Elapsed Time: 0:00:00 Time:  0:00:00


Accuracy on the training set for GAM with IV Binning = 0.7332
Accuracy on the test set for GAM with IV Binning = 0.7357


### Question 2
#### Re-train the generalized additive model in question 1 with the piecewise-linear feature engineering.
Use the pygam package, by setting the spline order equal to 1, p-spline model's basis function becomes piecewise-linear

In [71]:
x_train_PL = df_train_clean.iloc[:,1:].values
y_train_PL = df_train_clean.iloc[:,0].values

x_test_PL = df_test_clean.iloc[:,1:].values
y_test_PL = df_test_clean.iloc[:,0].values

x_train_PL = x_train_PL.reshape([-1,5])
n_splines=10


    
    # build P-spline, set spline_order to 1 so the basis function is piecewise linear
p_spl_PL = LogisticGAM(s(0, n_splines=n_splines,spline_order=1)\
                  +s(1, n_splines=n_splines, spline_order=1)\
                  +s(2, n_splines=n_splines, spline_order=1)\
                  +s(3, n_splines=n_splines, spline_order=1)\
                  +s(4, n_splines=n_splines, spline_order=1))
    #"0" the first explainatory variable(index of feature); n_spines df(dim of ); derivative specify the penalty
p_spl_PL.gridsearch(x_train_PL, y_train_PL) 
    #help you automatically pick lumda
    
    # predict
y_pred_train_PL = p_spl_PL.predict(x_train_PL)
y_pred_test_PL = p_spl_PL.predict(x_test_PL)

# accuracy score
accuracy_train_PL = accuracy_score(y_train_PL, y_pred_train_PL)
accuracy_test_PL = accuracy_score(y_test_PL, y_pred_test_PL)

print('Accuracy on the training set for GAM with piecewise-linear feature engineering = ', np.round(accuracy_train_PL,4))
print('Accuracy on the test set for GAM with piecewise-linear feature engineering = ', np.round(accuracy_test_PL,4))

100% (11 of 11) |########################| Elapsed Time: 0:00:01 Time:  0:00:01


Accuracy on the training set for GAM with piecewise-linear feature engineering =  0.7386
Accuracy on the test set for GAM with piecewise-linear feature engineering =  0.7404


### Question 3
#### Re-train the generalized additive model in question 1 with penalized B-splines .
Use the pygam package, by setting the spline order equal to 3, fit the p-spline model 

In [72]:
x_train_BS = df_train_clean.iloc[:,1:].values
y_train_BS = df_train_clean.iloc[:,0].values

x_test_BS = df_test_clean.iloc[:,1:].values
y_test_BS = df_test_clean.iloc[:,0].values

x_train_BS = x_train_BS.reshape([-1,5])
n_splines=8
spline_order=3

    
    # build P-spline
p_spl = LogisticGAM(s(0, n_splines=n_splines,spline_order=spline_order, penalties='derivative')\
                  +s(1, n_splines=n_splines,spline_order=spline_order, penalties='derivative')\
                  +s(2, n_splines=n_splines,spline_order=spline_order, penalties='derivative')\
                  +s(3, n_splines=n_splines,spline_order=spline_order, penalties='derivative')\
                  +s(4, n_splines=n_splines,spline_order=spline_order, penalties='derivative'))
    #"0" the first explainatory variable(index of feature); n_spines df(dim of ); derivative specify the penalty
p_spl.gridsearch(x_train_BS, y_train_BS) 
    #help you automatically pick lumda
    
    # predict
y_pred_train_BS = p_spl.predict(x_train_BS)
y_pred_test_BS = p_spl.predict(x_test_BS)

# accuracy score
accuracy_train_BS = accuracy_score(y_train_BS, y_pred_train_BS)
accuracy_test_BS = accuracy_score(y_test_BS, y_pred_test_BS)

print('Accuracy on the training set for GAM with penalized B-splines  =', np.round(accuracy_train_BS,4))
print('Accuracy on the test set for GAM with penalized B-splines  =', np.round(accuracy_test_BS,4))

100% (11 of 11) |########################| Elapsed Time: 0:00:01 Time:  0:00:01


Accuracy on the training set for GAM with penalized B-splines  = 0.7373
Accuracy on the test set for GAM with penalized B-splines  = 0.7409


### Question 4
#### Evaluate their model performance on the testing data and report the prediction accuracy. 

Prediction accuracy:

In [73]:
print('Accuracy on the test set for GAM with IV Binning =', np.round(accuracy_test_IV,4))
print('Accuracy on the test set for GAM with piecewise-linear feature engineering = ', np.round(accuracy_test_PL,4))
print('Accuracy on the test set for GAM with penalized B-splines  =', np.round(accuracy_test_BS,4))

Accuracy on the test set for GAM with IV Binning = 0.7357
Accuracy on the test set for GAM with piecewise-linear feature engineering =  0.7404
Accuracy on the test set for GAM with penalized B-splines  = 0.7409


#### Compare the models in (1), (2) and (3) in terms of model explainability. 
The prediction accuracy increases from 0.7357 to 0.7409 as the model grows more complicated.
However, the model explainability decreased during the process.

For the first model, GAM with IV Binning technology, it can be interpreted as how the predicted value will change if a variable falls within a certain range.

For GAM with piecewise-linear feature engineering and GAM with penalized B-splines (degree=3), it become harder for interpretation. Especially for penalized B-splines (degree=3), the non-linearity of basis function makes the model's overall much less explainable.


#### Draw your conclusions about the final model recommendation.

I would suggest the second model:  GAM with piecewise-linear feature engineering.
The prediction accuracy is a little bit too low for the first model compared to the other two models. 
GAM with piecewise-linear feature engineering model has only slightly lower prediction accuracy than the third one, but is much more explainable than GAM with penalized B-splines (degree=3). 


